In [1]:
import pandas as pd
import os
import glob
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyrogue.utilities.fileio as fileio
import numpy as np
import os
import time
import struct


In [2]:
# JSONファイルの読み込み
json_file_path = '2024c_aborts_detuned.json'

# JSONファイルを読み込んでDataFrameに変換
df = pd.read_json(json_file_path)


In [3]:
df

,bt_orbit_url,bunch,critical_trigger,diamond_url,diff_time_her_inj,diff_time_ler_inj,first_trigger,gap_wait_time,her_abort_bpm_out_url,her_abort_bpm_pos_url,...,ler_bor_url,ler_current,ler_diamond_dose,ler_lossmon_url,max_dose,origin_ring,pressure_burst,report,ring,timestamp
0,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:50:28.519918000
1,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:54:52.489282800
2,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:55:33.516413800
3,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:58:19.496898200
4,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:58:50.474553200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,48,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,LER,,,LER,2024-10-24 09:48:14.451730600
398,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,45,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,HER,,,HER,2024-10-24 11:32:33.565544300
399,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,45,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,LER,,,LER,2024-10-24 11:32:38.764101900
400,,393,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,19,1,,7,LER,,,LER,2024-10-24 15:55:22.955042400


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
df

,bt_orbit_url,bunch,critical_trigger,diamond_url,diff_time_her_inj,diff_time_ler_inj,first_trigger,gap_wait_time,her_abort_bpm_out_url,her_abort_bpm_pos_url,...,ler_bor_url,ler_current,ler_diamond_dose,ler_lossmon_url,max_dose,origin_ring,pressure_burst,report,ring,timestamp
0,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:50:28.519918000
1,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:54:52.489282800
2,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:55:33.516413800
3,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:58:19.496898200
4,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2249,Loss Monitor D10-3,,-999999,-999999,Loss Monitor D10-3,NaT,,,...,,0,0,,1128,HER,,,HER,2024-10-09 14:58:50.474553200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,48,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,LER,,,LER,2024-10-24 09:48:14.451730600
398,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,45,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,HER,,,HER,2024-10-24 11:32:33.565544300
399,,2346,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,45,4,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,7,LER,,,LER,2024-10-24 11:32:38.764101900
400,,393,Manual Abort,,-999999,-999999,Manual Abort,NaT,,,...,,19,1,,7,LER,,,LER,2024-10-24 15:55:22.955042400


# 1. detuned

In [8]:

df[ 
    (df['critical_trigger'] != 'Manual Abort') &
    (df['ring'] != 'HER') &
    (df['ler_current'] > 100)
]

,bt_orbit_url,bunch,critical_trigger,diamond_url,diff_time_her_inj,diff_time_ler_inj,first_trigger,gap_wait_time,her_abort_bpm_out_url,her_abort_bpm_pos_url,...,ler_bor_url,ler_current,ler_diamond_dose,ler_lossmon_url,max_dose,origin_ring,pressure_burst,report,ring,timestamp
39,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,Loss Monitor D4-2,,-999999,-999999,Loss Monitor D4-2,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,198,87,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,2,Both,"D04_L03,QDWOP.5",,Both,2024-10-10 19:55:34.109659000
41,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,CLAWS D05V1,,-999999,-999999,CLAWS D05V1,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,248,101,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,2,LER,,,LER,2024-10-10 21:12:23.656584100
42,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,Loss Monitor D4-2,,-999999,-999999,Loss Monitor D4-2,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,247,96,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,2,LER,"D04_L07,QW3OLP",,LER,2024-10-10 21:29:54.321409600
43,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,Loss Monitor D4-2,,-999999,-999999,Loss Monitor D4-2,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,249,87,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,2,LER,,,LER,2024-10-10 22:11:36.048739400
46,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,RF D05E,,-999999,-999999,RF D05E,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,348,64,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,2,LER,,,LER,2024-10-11 08:35:28.616757100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,CLAWS D05V1,,-999999,-999999,CLAWS D05V1,NaT,http://kekb-co-web.kek.jp/ScrShot_BT/HER/2024_...,http://kekb-co-web.kek.jp/ScrShot_BT/HER/2024_...,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,999,205,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,3,Both,"D10_L05,QDWNP.6,D10_L08,QW4NLP,D07_L02,FBL_kic...",,Both,2024-10-22 09:37:15.741991500
318,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,Loss Monitor D4-2,,-999999,-999999,Loss Monitor D4-2,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,999,163,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,64,LER,"D10_L05,QDWNP.6",,LER,2024-10-22 09:56:31.373145500
329,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,Loss Monitor D4-2,,-999999,-999999,Loss Monitor D4-2,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,996,62,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,58,LER,"D10_L02,QFWNP.4,D10_L08,QW4NLP",,LER,2024-10-22 10:40:44.102147400
373,http://kekb-co-web.kek.jp/doc/Image/BELLE/shot...,2346,CLAWS D05V1,,-999999,-999999,CLAWS D05V1,NaT,,,...,http://kekb-co-web.kek.jp/doc/Image/BELLE/abor...,971,52,http://www-linac2.kek.jp/kekb/scrshot1/2024_10...,18,LER,"D10_L08,QW4NLP",,LER,2024-10-23 02:21:43.817812400


In [6]:
def datfile(filePath):
    # Waveforms variables to be filled 
    ampFault = { i : []  for i in range(4) }  
    AbortFlagFault = []
    MA_UVFault = []
    MA_DVFault = []
    Pos_UVFault = []
    Pos_DVFault = []
    STD_UVFault = []
    STD_DVFault = []
    aborttime=[]
    mark=0
    
    # Open the .dat file
    with fileio.FileReader(files=filePath) as fd:
        for header,data in fd.records():
            if header.channel==27 or header.channel==28 or header.channel==29 or header.channel==30 :
                mark=1
        # Loop through the file data
        for header,data in fd.records():
            
            # Check if there is a 8-byte header in the frame
            if (header.flags==0):
                hdrOffset = 8
                hdr = struct.unpack("<d", data[:8])[0]
                timestamp = time.localtime(hdr)
                # Define the desired format string (replace with your preferred format)
                # Some common format specifiers:
                # %Y - Year with century
                # %m - Month as a decimal number (01-12)
                # %d - Day of the month as a decimal number (01-31)
                # %H - Hour in 24-hour format (00-23)
                # %M - Minute (00-59)
                # %S - Second (00-59)
                formatted_time = time.strftime('%Y-%m-%d %H:%M:%S', timestamp)
                print( f'CH={header.channel}: Event Timestamp in human-readable format: {formatted_time}' )
                if header.channel==24:
                    aborttime.append(formatted_time)
            else:
                hdrOffset = 0
                print( 'No timestamp header detected' )

            if mark==0:
                # Check for error in frame
                if (header.error>0):
                    # Look at record header data
                    print(f"Processing record. Total={fd.totCount}, Current={fd.currCount}")
                    print(f"Record size    = {header.size}")
                    print(f"Record channel = {header.channel}")
                    print(f"Record flags   = {header.flags:#x}")
                    print(f"Record error   = {header.error:#x}")
        
                # Check if AMP Fault waveform
                elif (header.channel < 16) and (header.channel >= 12):
                    ampFault[header.channel-12].append(data[hdrOffset:].view(np.int16))
        
                # Check if AMP Live waveform
                elif header.channel == 24:
                    fpData = data[hdrOffset:].view(np.float32)
                    fpDataLen = len(fpData)
                    AbortFlagFault.append(   fpData[0:(fpDataLen*3)+0:3] ) 
                    MA_UVFault.append(   fpData[1:(fpDataLen*3)+1:3] )
                    MA_DVFault.append( fpData[2:(fpDataLen*3)+2:3] )
        
                elif header.channel == 25:
                    fpData = data[hdrOffset:].view(np.float32)
                    fpDataLen = len(fpData)
                    Pos_UVFault.append(   fpData[1:(fpDataLen*3)+1:3] )
                    Pos_DVFault.append( fpData[2:(fpDataLen*3)+2:3] )
        
                elif header.channel == 26:
                    fpData = data[hdrOffset:].view(np.float32)
                    fpDataLen = len(fpData)
                    STD_UVFault.append(   fpData[1:(fpDataLen*3)+1:3] )
                    STD_DVFault.append( fpData[2:(fpDataLen*3)+2:3] )
                
                # Else undefined stream index
                else:
                    print( f'UNDEFINED DATA STREAM[{header.channel}]!!!')
            elif mark==1:
                # Check for error in frame
                if (header.error>0):
                    # Look at record header data
                    print(f"Processing record. Total={fd.totCount}, Current={fd.currCount}")
                    print(f"Record size    = {header.size}")
                    print(f"Record channel = {header.channel}")
                    print(f"Record flags   = {header.flags:#x}")
                    print(f"Record error   = {header.error:#x}")
    
                # Check if AMP Fault waveform
                elif (header.channel < 16) and (header.channel >= 12):
                    ampFault[header.channel-12].append(data[hdrOffset:].view(np.int16))
        
                # Check if AMP Live waveform
                elif header.channel == 24:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    Pos_UVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 25:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    Pos_DVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 26:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    MA_UVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 27:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    MA_DVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 28:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    STD_UVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 29:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    STD_DVFault.append(   fpData[:fpDataLen] )
                elif header.channel == 30:
                    fpData = data[hdrOffset:].view(np.float16)
                    fpDataLen = len(fpData)
                    AbortFlagFault.append(   fpData[:fpDataLen] )
        return ampFault, AbortFlagFault, MA_UVFault, MA_DVFault, Pos_UVFault, Pos_DVFault, STD_UVFault, STD_DVFault, aborttime, mark

In [9]:
def parse_and_save(dat_file):
    ampFault, recordtime=datfile(dat_file)
    print(recordtime)
    for i in range(len(recordtime)):
        given_time = datetime.strptime(recordtime[i], '%Y-%m-%d_%H-%M-%S')
        start_time = given_time - timedelta(seconds=10)
        end_time = given_time + timedelta(seconds=10)

        filtered_df = df[(df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)
                        & (df['critical_trigger'] != 'Manual Abort') 
                        & (df['ring'] != 'HER')
                        & (df['ler_current'] > 60)
                        ]
        
        if not filtered_df.empty:
            print(np.array(filtered_df.to_numpy()[0]))
            #np.savez_compressed(f'/mnt/SBOR/2024ab_RFSoC/ZCU111/Old_Firmware/LERFUJI_{recordtime[i]}.npz', Basic=filtered_df.to_numpy()[0], DV_sum=ampFault[0][i], DV_delta=ampFault[1][i], UV_sum=ampFault[2][i], UV_delta=ampFault[3][i])
        else:
            print("該当する行はありません。")
        
        

def process_dat_file(dat_file_path):
    # ここに.datファイルに対する処理を記述する
    print("Processing:", dat_file_path)
    parse_and_save(dat_file_path)

# ディレクトリのパス
directory_path = "/home/nomaru/projects/kek-bpm-rfsoc-dev/software/datfile/"

# 指定した日付
start_date = datetime(2024, 3, 7, 0, 0, 0)
end_date = datetime(2024, 3, 8, 23, 59, 59)

# ディレクトリ内のすべての.datファイルに対して処理を実行
for dat_file in glob.glob(os.path.join(directory_path, "*.dat")):
    # ファイル名から日付部分を抽出
    match = re.search(r'data_(\d{8})_(\d{6})\.dat', dat_file)
    if match:
        file_datetime = datetime.strptime(match.group(1) + match.group(2), '%Y%m%d%H%M%S')
        # 指定した日付以降のファイルのみ処理する
        if end_date >= file_datetime >= start_date:
            process_dat_file(dat_file)
                


Processing: /home/nomaru/projects/kek-bpm-rfsoc-dev/software/datfile/data_20240308_055253.dat
['2024-03-08_13-45-43', '2024-03-08_13-46-55', '2024-03-08_15-22-06', '2024-03-08_15-30-00', '2024-03-08_17-05-08', '2024-03-08_20-08-37', '2024-03-08_22-05-09']
['http://kekb-co-web.kek.jp/doc/Image/BELLE/shotID/png/20240308-134543_BothAbort_Belle2VXDdiamond.png'
 783 'Belle2 VXD diamond'
 'http://kekb-co-web.kek.jp/doc/Image/BELLE/diamond_abort/png/2024-03-08_13-45-43_99969.png'
 -999999 -999999 'Belle2 VXD diamond' NaT '' '' '' '' 15 0
 'http://www-linac2.kek.jp/kekb/scrshot1/2024_03/08/2024_03_08_13_45_58.png'
 '' ''
 'http://kekb-co-web.kek.jp/ScrShot_BT/LER/2024_03/08/20240308_1345Output.png'
 'http://kekb-co-web.kek.jp/ScrShot_BT/LER/2024_03/08/20240308_1345Position_XY.png'
 'http://kekb-co-web.kek.jp/ScrShot_BT/LER/2024_03/08/20240308_1345Position_Time.png'
 'http://kekb-co-web.kek.jp/doc/Image/BELLE/abort_summary/web/BOR/20240308_134555_LER.png'
 196 122
 'http://www-linac2.kek.jp/kek